## Finetuning Open Source model using PEFT, LoRA, 4Bit quantization, and TRL!

The goal of this notebook is to experiment with a new way to make it very easy to build a task-specific model for your use-case.

To create your model, just go to the first code cell, and describe the model you want to build in the prompt. Be descriptive and clear.

Select a temperature (high=creative, low=precise), and the number of training examples to generate to train the model. From there, just run all the cells.

You can change the model you want to fine-tune by changing `model_name` in the `Define Hyperparameters` cell.

Split into train and test sets.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/mkthoma/llm_finetuning.git

Cloning into 'llm_finetuning'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 17 (delta 2), reused 13 (delta 2), pack-reused 0
Receiving objects: 100% (17/17), 16.32 MiB | 7.80 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Filtering content: 100% (2/2), 59.75 MiB | 11.16 MiB/s, done.


In [ ]:
# Training and validation datasaet from HF Datasets
# https://huggingface.co/datasets/OpenAssistant/oasst1?row=1
import pandas as pd
from datasets import load_dataset

# Load datasets
train_dataset = load_dataset('json', data_files='/content/llm_finetuning/finetuning_data_train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='/content/llm_finetuning/finetuning_data_val.jsonl', split="train")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 20587
})

In [ ]:
valid_dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 1095
})

# Install necessary libraries

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


# Define Hyperparameters

In [ ]:
model_name = "microsoft/phi-2"
dataset_name = "finetuning_data_train.jsonl"
new_model = "phi_2_finetuned"

lora_r = 64
lora_alpha = 16
lora_dropout = 0.05
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant=True
# use_nested_quant = False

compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

output_dir = "./outputs"
num_train_epochs = 1
fp16 = True
bf16 = False
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 1
gradient_checkpointing = False
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
group_by_length = True
logging_steps=5000
logging_strategy="steps"
max_seq_length = 512
packing = False
device_map = {"": 0}


################################## newly added
save_strategy="epoch"


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True)

model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules=['Wqkv','out_proj'],
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=True,
    bf16=False,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=False,
    # save_strategy=save_strategy,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5000  # Evaluate every n steps
)


model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Load Datasets and Train

In [ ]:
# Preprocess datasets
system_message = "You are a question answering chatbot. Provide a clear and detailed explanation"
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [f'[INST] <<SYS>>\n{system_message.strip()}\n<</SYS>>\n\n' + prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()
trainer.model.save_pretrained(new_model)

# Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n Suggest some food dishes [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(prompt)
print(result[0]['generated_text'])

Map:   0%|          | 0/20587 [00:00<?, ? examples/s]

Map:   0%|          | 0/1095 [00:00<?, ? examples/s]

Map:   0%|          | 0/20587 [00:00<?, ? examples/s]

Map:   0%|          | 0/1095 [00:00<?, ? examples/s]

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
5000,1.582800,1.490877
10000,1.540500,1.475482


Step,Training Loss,Validation Loss
5000,1.582800,1.490877
10000,1.540500,1.475482
15000,1.535300,1.468661
20000,1.516800,1.461401


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[INST] <<SYS>>
You are a question answering chatbot. Provide a clear and detailed explanation
<</SYS>>

 Suggest some food dishes [/INST] Sure, here are some food dishes that you can try:

1. Tacos: Tacos are a popular Mexican dish that consists of a tortilla filled with various ingredients such as meat, vegetables, and cheese.

2. Pizza: Pizza is an Italian dish that consists of a flatbread topped with tomato sauce, cheese, and various toppings such as pepperoni, mushrooms, and olives.

3. Sushi: Sushi is a Japanese dish that consists of vinegared rice and various toppings such as raw fish, avocado, and cucumber.

4. Pad Thai: Pad Thai is a Thai dish that consists of stir-fried noodles, vegetables, and peanuts.

5. Curry: Curry is a dish that is popular in many countries and consists of


# Run Inference

In [ ]:
from transformers import pipeline

prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n What is an LLM? [/INST]" # replace the command here with something relevant to your task
num_new_tokens = 500  # change to the number of new tokens you want to generate

# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])

# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

 An LSTM (Long Short Term Memory) is a type of recurrent neural network (RNN) that is commonly used in natural language processing (NLP) tasks such as text generation, language translation, and sentiment analysis.

Unlike traditional RNNs, which have a fixed number of hidden layers, LSTMs have a variable number of hidden layers and gates that allow them to capture long-term dependencies in the input data. This makes them particularly well-suited for processing sequential data such as text and speech.

The basic architecture of an LSTM consists of a stack of cells, each of which has a memory cell and an input gate. The memory cell stores the state of the LSTM over time, while the input gate controls the flow of information into the LSTM. The output gate determines whether the LSTM should produce a new output based on the current state and input.

During training, the LSTM is fed a sequence of inputs and its output is compared to the desired output. The weights of the LSTM are adjusted t

# Merge the model and save

In [ ]:
# Merge and save the fine-tuned model
model_path = "/content/drive/MyDrive/finetuned_phi2"  # change to your preferred path

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

The repository for microsoft/phi-2 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('/content/drive/MyDrive/finetuned_phi2/tokenizer_config.json',
 '/content/drive/MyDrive/finetuned_phi2/special_tokens_map.json',
 '/content/drive/MyDrive/finetuned_phi2/vocab.json',
 '/content/drive/MyDrive/finetuned_phi2/merges.txt',
 '/content/drive/MyDrive/finetuned_phi2/added_tokens.json',
 '/content/drive/MyDrive/finetuned_phi2/tokenizer.json')

# Load a fine-tuned model from Drive and run inference

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "/content/drive/MyDrive/finetuned_phi2"  # change to your preferred path

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import pipeline

system_message = "You are a question answering chatbot. Provide a clear and detailed explanation"
question = "How to calm down a person?"
prompt = f"[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n {question} [/INST]" # replace the command here with something relevant to your task

num_new_tokens = 500  # change to the number of new tokens you want to generate
# Count the number of tokens in the prompt
num_prompt_tokens = len(tokenizer(prompt)['input_ids'])
# Calculate the maximum length for the generation
max_length = num_prompt_tokens + num_new_tokens

gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=max_length)
result = gen(prompt)
print(result[0]['generated_text'].replace(prompt, ''))

The model 'PhiForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausa

 There are several ways to calm down a person, depending on the situation and the person's needs. Here are some general tips:

1. Listen actively: Allow the person to express their feelings and concerns without interruption. Show empathy and understanding, and validate their emotions.

2. Use calming techniques: Encourage the person to use calming techniques such as deep breathing, progressive muscle relaxation, or visualization. These techniques can help reduce anxiety and promote relaxation.

3. Provide reassurance: Reassure the person that they are safe and that their feelings are valid. Let them know that you are there to support them and that you will help them through the situation.

4. Offer practical solutions: If the person is experiencing a specific problem, offer practical solutions or suggestions to help them address the issue. This can help them feel more in control and reduce their anxiety.

5. Encourage physical activity: Physical activity can help reduce stress and anxi